This file contains Bert

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
np.random.seed(42) # results are reproducible and comparable across different runs

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df=pd.read_csv('/content/drive/MyDrive/fake_papers_train_part_public.csv')
df = pd.read_csv("/content/fake_papers_train_part_public.csv")
df = df.drop(columns = ['id'], axis = 1)
print(df.shape)
df.head()

(5350, 2)


,text,fake
0,Modern two-dimensional imaging is of such qual...,0
1,Background: The optimal sequence of systemic p...,1
2,This chapter opens with a discussion of the ef...,1
3,The time scale of the ultra-short-term can str...,1
4,Electronic nose or machine olfaction are syste...,1


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
# Load pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
df.groupby('fake').describe()

text                                                               
     count unique                                                top freq
fake                                                                     
0     1686   1686  Modern two-dimensional imaging is of such qual...    1
1     3664   3664  Background: The optimal sequence of systemic p...    1

In [ ]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
1686/3664

0.4601528384279476

In [ ]:
df_bot = df[df['fake']==1]
df_bot.shape

(3664, 2)

In [ ]:
df_human = df[df['fake']==0]
df_human.shape

(1686, 2)

In [ ]:
df_bot_downsampled = df_bot.sample(df_human.shape[0])
df_bot_downsampled.shape

(1686, 2)

In [ ]:
df_balanced = pd.concat([df_bot_downsampled, df_human])
df_balanced.shape

(3372, 2)

In [ ]:
df_balanced['fake'].value_counts()

1    1686
0    1686
Name: fake, dtype: int64

In [ ]:
df_balanced[0]=df_balanced['fake'].apply(lambda x: 1 if x==0 else 0)
df_balanced.sample(5)

,text,fake,0
4457,"In this chapter, the focus of the paper is on ...",1,0
2110,"In this paper, the authors examine the effects...",1,0
3865,This paper presents an efficient and optimized...,0,1
4681,"In this paper, the authors discuss the require...",1,0
4464,"In this paper, the authors describe a new ener...",1,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced[0], stratify=df_balanced[0])

In [ ]:
X_train.head(4)

774     In various parts of the world, but especially ...
3259    This is a very short paper. It describes a new...
2601    The aim of this study was to investigate the o...
4529    Because of the clinical significance of caroti...
Name: text, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
80/80 [==============================] - 1162s 15s/step - loss: 0.6748 - accuracy: 0.5868 - precision: 0.5879 - recall: 0.5818
Epoch 2/3
80/80 [==============================] - 1159s 14s/step - loss: 0.6423 - accuracy: 0.6327 - precision: 0.6235 - recall: 0.6704
Epoch 3/3
80/80 [==============================] - 1161s 15s/step - loss: 0.6230 - accuracy: 0.6520 - precision: 0.6619 - recall: 0.6221


In [ ]:
model.evaluate(X_test, y_test)

27/27 [==============================] - 388s 14s/step - loss: 0.6443 - accuracy: 0.6002 - precision: 0.5607 - recall: 0.9216


[0.6442532539367676, 0.600237250328064, 0.560693621635437, 0.9216151833534241]

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

27/27 [==============================] - 384s 14s/step


In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

array([[118, 304],
       [ 33, 388]])

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.78      0.28      0.41       422
           1       0.56      0.92      0.70       421

    accuracy                           0.60       843
   macro avg       0.67      0.60      0.55       843
weighted avg       0.67      0.60      0.55       843

